In [1]:
from NER_cadec import training_loop
model = training_loop("./datasets/cadec", verbose=True)

/Users/simonelusetti/phd/NER/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Batch loss 0.4227069616317749
Batch loss 0.2678055167198181
Batch loss 0.22630029916763306
Batch loss 0.23801881074905396
Batch loss 0.1744709610939026
Batch loss 0.21314378082752228
Batch loss 0.24280136823654175
Batch loss 0.2231641709804535
Batch loss 0.20719073712825775
Batch loss 0.1650230437517166
Batch loss 0.21489082276821136
Batch loss 0.19373521208763123
Batch loss 0.2070760726928711
Batch loss 0.1753310114145279
Batch loss 0.19897152483463287
Batch loss 0.18144837021827698
Batch loss 0.12118695676326752
Batch loss 0.13534528017044067
Batch loss 0.1500398814678192
Batch loss 0.12393879145383835
Epoch 1, Loss: 0.2041294787079096
Batch loss 0.16393642127513885
Batch loss 0.1275119036436081
Batch loss 0.17660117149353027
Batch loss 0.13980796933174133
Batch loss 0.12600184977054596
Batch loss 0.1369914561510086
Batch loss 0.14825572073459625
Batch loss 0.13809575140476227
Batch loss 0.13833661377429962
Batch loss 0.14033210277557373
Batch loss 0.14449971914291382
Batch loss 0.11

In [2]:
import torch
torch.save(model.state_dict(), "./datasets/cadec/model.pth")